## Refernce
[Paper](https://www.sciencedirect.com/science/article/abs/pii/S0010482518302713) <br>
[Link](https://github.com/tom-beer/Arrhythmia-CNN)

In [28]:
# from google.colab import drive
# drive.mount('/content/drive')

In [29]:
# !git clone https://github.com/tom-beer/Arrhythmia-CNN.git
# %cd Arrhythmia-CNN/

In [30]:
from __future__ import print_function
import torch
import torch.utils.data
import numpy as np
import pandas as pd

from torch import nn, optim
from torch.utils.data.dataset import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [31]:
is_cuda = True
num_epochs = 100
batch_size = 10
torch.manual_seed(46)
log_interval = 10
in_channels_ = 1
num_segments_in_record = 100
segment_len = 3600
num_records = 48
num_classes = 16
allow_label_leakage = True

device = torch.device("cuda:0" if is_cuda else "cpu")

In [ ]:
# train_ids, test_ids = train_test_split(np.arange(index_set), train_size=.8, random_state=46)
# scaler = MinMaxScaler(feature_range=(0, 1), copy=False)

class CustomDatasetFromCSV(Dataset):
    def __init__(self, data_path, transforms_=None):
        self.df = pd.read_pickle(data_path)
        self.transforms = transforms_

    def __getitem__(self, index):
        row = self.df.iloc[index]
        signal = row['signal']
        target = row['target']
        if self.transforms is not None:
            signal = self.transforms(signal)
        signal = signal.reshape(1, signal.shape[0])
        return signal, target

    def __len__(self):
        return self.df.shape[0]


train_dataset = CustomDatasetFromCSV('/content/drive/MyDrive/Arrhythmia-CNN-master/data/Arrhythmia_dataset.pkl')
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = CustomDatasetFromCSV('/content/drive/MyDrive/Arrhythmia-CNN-master/data/Arrhythmia_dataset.pkl')
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)


class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)


def basic_layer(in_channels, out_channels, kernel_size, batch_norm=False, max_pool=True, conv_stride=1, padding=0
                , pool_stride=2, pool_size=2):
    layer = nn.Sequential(
        nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=conv_stride,
                  padding=padding),
        nn.ReLU())
    if batch_norm:
        layer = nn.Sequential(
            layer,
            nn.BatchNorm1d(num_features=out_channels))
    if max_pool:
        layer = nn.Sequential(
            layer,
            nn.MaxPool1d(kernel_size=pool_size, stride=pool_stride))

    return layer


class arrhythmia_classifier(nn.Module):
    def __init__(self, in_channels=in_channels_):
        super(arrhythmia_classifier, self).__init__()
        self.cnn = nn.Sequential(
            basic_layer(in_channels=in_channels, out_channels=128, kernel_size=50, batch_norm=True, max_pool=True,
                        conv_stride=3, pool_stride=3),
            basic_layer(in_channels=128, out_channels=32, kernel_size=7, batch_norm=True, max_pool=True,
                        conv_stride=1, pool_stride=2),
            basic_layer(in_channels=32, out_channels=32, kernel_size=10, batch_norm=False, max_pool=False,
                        conv_stride=1),
            basic_layer(in_channels=32, out_channels=128, kernel_size=5, batch_norm=False, max_pool=True,
                        conv_stride=2, pool_stride=2),
            basic_layer(in_channels=128, out_channels=256, kernel_size=15, batch_norm=False, max_pool=True,
                        conv_stride=1, pool_stride=2),
            basic_layer(in_channels=256, out_channels=512, kernel_size=5, batch_norm=False, max_pool=False,
                        conv_stride=1),
            basic_layer(in_channels=512, out_channels=128, kernel_size=3, batch_norm=False, max_pool=False,
                        conv_stride=1),
            Flatten(),
            nn.Linear(in_features=1152, out_features=512),
            nn.ReLU(),
            nn.Dropout(p=.1),
            nn.Linear(in_features=512, out_features=num_classes),
            nn.Softmax()
        )

    def forward(self, x, ex_features=None):
        return self.cnn(x)


def calc_next_len_conv1d(current_len=112500, kernel_size=16, stride=8, padding=0, dilation=1):
    return int(np.floor((current_len + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1))


model = arrhythmia_classifier().to(device).double()
lr = 3e-1
num_of_iteration = len(train_dataset) // batch_size

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)
criterion = nn.NLLLoss()


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        all_=0
        t_=0
        for batch_idx, (data, target) in enumerate(test_loader):
            all_+=1
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()

            if batch_idx == 0:
                n = min(data.size(0), 4)

            list_=output[0].tolist()
            pridict_=int(list_.index(max(list_)))
            if int(target)==pridict_:
                t_+=1
    accuracy_=t_/all_
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.5f}'.format(test_loss))
    print('====> Test Accuracy: {:.5f}'.format(accuracy_))
    # print(f'Learning rate: {optimizer.param_groups[0]["lr"]:.6f}')
for epoch in range(1, num_epochs + 1):
  train(epoch)
  test(epoch)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1 [0/823 (0%)]	Loss: -0.006458
Train Epoch: 1 [100/823 (12%)]	Loss: -0.100000
Train Epoch: 1 [200/823 (24%)]	Loss: -0.100000
Train Epoch: 1 [300/823 (36%)]	Loss: 0.000000
Train Epoch: 1 [400/823 (48%)]	Loss: 0.000000
Train Epoch: 1 [500/823 (60%)]	Loss: 0.000000
Train Epoch: 1 [600/823 (72%)]	Loss: 0.000000
Train Epoch: 1 [700/823 (84%)]	Loss: 0.000000
Train Epoch: 1 [800/823 (96%)]	Loss: 0.000000
====> Epoch: 1 Average loss: -0.0332
====> Test set loss: -0.34386
====> Test Accuracy: 0.34386
Train Epoch: 2 [0/823 (0%)]	Loss: -0.100000
Train Epoch: 2 [100/823 (12%)]	Loss: -0.100000
Train Epoch: 2 [200/823 (24%)]	Loss: -0.100000
Train Epoch: 2 [300/823 (36%)]	Loss: -0.000000
Train Epoch: 2 [400/823 (48%)]	Loss: -0.000000
Train Epoch: 2 [500/823 (60%)]	Loss: -0.000000
Train Epoch: 2 [600/823 (72%)]	Loss: -0.000000
Train Epoch: 2 [700/823 (84%)]	Loss: -0.000000
Train Epoch: 2 [800/823 (96%)]	Loss: -0.000000
====> Epoch: 2 Average loss: -0.0344
====> Test set loss: -0.34386
===